In [1]:

import os
os.environ['PYTENSOR_FLAGS'] = 'floatX=float32'
import numpy as np

from pcntoolkit.normative_model.norm_utils import norm_init
from pcntoolkit.util.utils import simulate_data
import matplotlib.pyplot as plt
from pcntoolkit.normative import estimate
from warnings import filterwarnings
filterwarnings('ignore')
import pymc as pm
import pytensor
import cProfile
import timeit
import pickle


In [2]:


########################### Experiment Settings ###############################


random_state = 29

working_dir = '/home/guus/tmp'  # Specify a working directory to save data and results.

simulation_method = 'linear'
n_features = 1      # The number of input features of X
n_grps = 10          # Number of batches in data
n_samples = 2500     # Number of samples in each group (use a list for different
# sample numbers across different batches)

model_type = 'bspline' #  modelto try 'linear, ''polynomial', 'bspline'   



In [3]:


# ############################## Data Simulation ################################


# X_train, Y_train, grp_id_train, X_test, Y_test, grp_id_test, coef = \
#     simulate_data(simulation_method, n_samples, n_features, n_grps,
#                   working_dir=working_dir, plot=True, noise='heteroscedastic_nongaussian', 
#                   random_state=random_state)


In [4]:
# train_sex_mask = np.random.choice([0, 1], size=X_train.shape[0])
# Y_train[train_sex_mask == 0] += 0.2
# grp_id_train = np.stack([grp_id_train[:,0], train_sex_mask],axis=1)
# print(grp_id_train.shape)


# test_sex_mask = np.random.choice([0, 1], size=X_test.shape[0])
# Y_test[test_sex_mask == 0] += 0.2
# grp_id_test = np.stack([grp_id_test[:,0], test_sex_mask],axis=1)
# print(grp_id_test.shape)


In [5]:
# savedir = "/home/guus/Desktop/pcn_profile_data"
# with open(os.path.join(savedir, 'train_data'), 'wb') as f:
#     pickle.dump((X_train, Y_train, grp_id_train), f)
# with open(os.path.join(savedir, 'test_data'), 'wb') as f:
#     pickle.dump((X_test, Y_test, grp_id_test), f)
    

In [6]:
savedir = "/home/guus/Desktop/pcn_profile_data"


In [7]:

def load_data():
    import pickle
    with open(os.path.join(savedir, 'train_data'), 'rb') as f:
        X_train, Y_train, grp_id_train = pickle.load(f)
    with open(os.path.join(savedir, 'test_data'), 'rb') as f:
        X_test, Y_test, grp_id_test = pickle.load(f)
    return X_train, Y_train, grp_id_train, X_test, Y_test, grp_id_test

In [8]:
X_train, Y_train, grp_id_train, X_test, Y_test, grp_id_test = load_data()

In [9]:

################################# Fittig and Predicting ###############################
model_confs = {"M4":{"random_intercept_mu":'True',"likelihood":"SHASHb"}}

# model_confs = {"M1":{"random_intercept_mu":'False',"likelihood":"Normal"},
#                "M2":{"random_intercept_mu":'True',"likelihood":"Normal"},
#                "M3":{"random_intercept_mu":'True',"likelihood":"SHASHo"},
#                "M4":{"random_intercept_mu":'True',"likelihood":"SHASHb"}}

for model_name, model_conf in model_confs.items():
    print(f"Model: {model_name}")
    print("Initializing the model")
    nm = norm_init(X_train, Y_train, 
               alg='hbr',
               model_type=model_type,
               linear_mu='True',
               random_slope_mu='False',
               linear_sigma='False',
               linear_epsilon='False',
               linear_delta='False',
               **model_conf)
    
    os.makedirs(os.path.join(savedir, 'v30', model_name), exist_ok=True)

    print("Getting the model")
    model = nm.hbr.get_model(X_train, Y_train, grp_id_train)

    print("Profiling logp")
    logp_profile = model.profile(model.logp())
    print("Saving profile as text")
    with open(os.path.join(savedir, 'v30', model_name, 'logp_profile.txt'), 'wt') as a:
        logp_profile.summary(a)
    print("Saving profile as pickle")
    with open(os.path.join(savedir, 'v30', model_name, 'logp_profile.pkl'), 'wb') as a:
        pickle.dump(logp_profile, a)


    print("Profiling gradient")
    grad_profile = model.profile(pm.gradient(model.logp())) 
    print("Saving profile as text")
    with open(os.path.join(savedir, 'v30', model_name, 'grad_profile.txt'), 'wt') as a:
        grad_profile.summary(a)
    print("Saving profile as pickle")
    with open(os.path.join(savedir, 'v30', model_name, 'grad_profile.pkl'), 'wb') as a:
        pickle.dump(grad_profile, a)
        
    print("Saving visualization of logp")
    pytensor.printing.pydotprint(model.logp(), os.path.join(savedir,'v30', model_name, 'logp.png'), var_with_name_simple=True)
    print("Saving visualization of logp gradient")
    pytensor.printing.pydotprint(pm.gradient(model.logp()), os.path.join(savedir,'v30', model_name, 'logp_grad.png'), var_with_name_simple=True)

Model: M4
Initializing the model
Getting the model
Profiling logp
Saving profile as text
Saving profile as pickle
Profiling gradient
Saving profile as text
Saving profile as pickle
Saving visualization of logp
The output file is available at /home/guus/Desktop/pcn_profile_data/v30/M4/logp.png
Saving visualization of logp gradient
The output file is available at /home/guus/Desktop/pcn_profile_data/v30/M4/logp_grad.png


In [ ]:

nm = norm_init(X_train, Y_train, 
               alg='hbr',
               model_type=model_type,
               likelihood='Normal',
               linear_mu='True',
               random_intercept_mu='False',
               random_slope_mu='False',
               linear_sigma='False',
               linear_epsilon='False',
               linear_delta='False')


In [10]:
model = nm.hbr.get_model(X_train, Y_train, grp_id_train)

In [15]:

grad_profile = model.profile(pm.gradient(model.logp())) 

with open('grad_profile.txt', 'wt') as a:
    grad_profile.summary(a)

In [8]:
print(type(model.logp()))

<class 'pytensor.tensor.variable.TensorVariable'>


In [9]:
pytensor.printing.pydotprint(model.logp(), outfile="model_logp.png", var_with_name_simple=True)

The output file is available at model_logp.png


In [53]:
a = np.random.randn(50,50)
timeit.timeit(lambda : a.reshape(-1), number=1000000)


0.30615136500273366

In [34]:
timeit.timeit(lambda : np.reshape(a, -1), number=1000000)


2.054333816999133

In [10]:
np.reshape(a, -1)

array([ 0.17341095, -0.2507605 ,  2.1881623 , ...,  0.78394736,
        0.21933634, -0.60980084])